本文主要记录chapter6-实战指南中visdom用于可视化的使用方法：

首先，作者定义了一个visualizer类用于封装visdom的基本操作（我也不知道为什么这么做，我个人觉得直接调用函数更简单，可能是这样把所有显示的函数集中在一起更好查看吧）

在这个类中定义了显示图片散点以及打印信息的方法。

我要记录的问题主要有哪些：

- 在一个程序里，我主要是记录哪些信息
- 这些信息通过哪些方式去完成记录，如何输出到visdom中
- 如何根据visdom的输出效果调参

## 在深度学习程序里，我需要记录哪些信息

记录的信息有loss,若干个个batch输出一次；val计算出的accuracy,test计算出的accuracy；还有当前的loss或者val的 accuracy对应的batch号

## 这些信息通过哪些方式去完成记录

- loss可以记录在一个列表里，作者用的方法是用torchnet的一个class：meter，这个可以通过调用meter.AverageValueMeter用列表的方式输入loss，通过调用value输出均值（方差），通过reset方法清零。(这种方法的好处应该就是可以输出多张类型的数据比如均值方差都可以)
```python
loss_meter = meter.AverageValueMeter()
loss_meter.reset()
loss_meter.add(loss.data[0])
if ii%opt.print_freq==opt.print_freq-1:
    vis.plot('loss', loss_meter.value()[0])
```

- val_accuracy是通过混淆矩阵来计算的，作者同样用的方法是用torchnet的一个class：meter，通过调用meter.ConfusionMeter()将预测结果与标签值输入meter.ConfusionMeter()，之后计算出混淆矩阵（即真psoitive，假psoitive，真nagetive,假nagetive组成的矩阵）PS:这里我想不明白为什么要用meter这个类。
```python
cm_value = confusion_matrix.value()
confusion_matrix.add(score.data, target.data)
val_accuracy = 100. * (cm_value[0][0] + cm_value[1][1]) / (cm_value.sum())
vis.plot('val_accuracy',val_accuracy)
```
- loss也可以直接存在一个数里，每n次输出一下就可以了，多分类问题也可以取max之后与label比较计算precsion.

```python
 running_loss += loss.data[0]
        if i % 2000 == 1999: # 每2000个batch打印一下训练状态
            print('[%d, %5d] loss: %.3f' \
                  % (epoch+1, i+1, running_loss / 2000))
            running_loss = 0.0
```
```python
_, predicted = t.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
```

In [4]:
def __init__(self, env='default', **kwargs):
        self.vis = visdom.Visdom(env=env, **kwargs)
        
        # 画的第几个数，相当于横座标
        # 保存（’loss',23） 即loss的第23个点
        self.index = {} 
        self.log_text = ''
def plot(self, name, y,**kwargs):
        '''
        self.plot('loss',1.00)
        '''
        x = self.index.get(name, 0)#检索index 字典里是否有存储‘loss’的值，没有的话，设为0
        self.vis.line(Y=np.array([y]), X=np.array([x]),#
                      win=name,
                      opts=dict(title=name),
                      update=None if x == 0 else 'append',
                      **kwargs
                      )
        self.index[name] = x + 1

### 总结：
总而言之，loss是直接计算出来，每n次计算一次均值，之后把loss均值以及本次loss均值对应的batch号输入到 visdom.Visdom.line里就可以了
val_accuracy同样,只需输入val_accuracy以及对应的对应的batch号，只不过可以利用混淆矩阵计算accuracy(把真正例与真反例都算作分类正确的结果)

我写的话，还是把作者的visualize.py拷贝过来吧，调用plot就可以了。要不然就是在epoch外面设置batch编号或者在函数内部设置一个静态变量作为batch变量（等晚上看看GPU可以用了之后就自己试一下）

### 之后如何使用：
- 把visualize.py文件放到主文件夹的utils子文件夹中，加上__init__.py文件中间加上from .visualize import Visualizer
- 之后，主文件中```from utils.visualize import Visualizer```
- 之后：```vis = Visualizer(opt.env)```配置visdom的环境
- 计算完loss 或val_accuracy之后，将数值存储起来，固定周期计算一下均值，与当前batch编号一起输出到visdom中
```python
vis.plot('loss', loss_meter.value()[0])```
或
```vis.plot('val_accuracy',val_accuracy)```
即可

## 3如何根据结果调参
### 调参之前应该做的事

1. 可视化训练过程中每个step（batch）的loss。如果是分类任务，可以顺便可视化出每个batch的准确率（不均衡数据可视化F1-score）。
2. 将训练日志在打印到屏幕上的同时也写入到本地磁盘。如果能实时同步写入那更好了（在python中可以用logging模块可以轻松实现。一个handler输出到屏幕，再设置一个handler输出到磁盘即可）。
3. 借助tensorflow里的FLAGS模块或者python-fire工具将你的训练脚本封装成命令行工具。
4. 代码中完成tensorboard等训练过程可视化环境的配置，最少要可视化出训练loss曲线。
5. 如果使用tensorflow，记得设置GPU内存动态增长（除非你只有一个GPU并且你确信一个训练任务会消耗GPU的一大半显存）
6. 初始调参阶段记得关闭L2、Dropout等用来调高模型泛化能力的超参数呐，它们很可能极大的影响loss曲线，干扰你的重要超参数的选取。然后根据自己的任务的量级，预估一个合理的batch size（一般来说64是个不错的初始点。数据集不均衡的话建议使用更大一点的值，数据集不大模型又不是太小的情况下建议使用更小一些的值）。如果对网络参数的随机初始化策略缺乏经验知识（知识来源于相关任务的论文实验细节或开源项目等），可以使用He方法[1]（使用ReLU激活时）或Xavier方法[2]来进行网络参数初始化。

1. 方法见visdom_exercise文件
2. 待学
3. 
在讲解主程序`main.py`之前，我们先来看看2017年3月谷歌开源的一个命令行工具`fire`[^3] ，通过`pip install fire`即可安装。下面来看看`fire`的基础用法，假设`example.py`文件内容如下：

```python
import fire

def add(x, y):
  return x + y
  
def mul(**kwargs):
    a = kwargs['a']
    b = kwargs['b']
    return a * b

if __name__ == '__main__':
  fire.Fire()
```

那么我们可以使用：

```bash
python example.py add 1 2 # 执行add(1, 2)
python example.py mul --a=1 --b=2 # 执行mul(a=1, b=2), kwargs={'a':1, 'b':2}
python example.py add --x=1 --y==2 # 执行add(x=1, y=2)
```

可见，只要在程序中运行`fire.Fire()`，即可使用命令行参数`python file <function> [args,] {--kwargs,}`。fire还支持更多的高级功能，具体请参考官方指南[^4] 。

[^3]: https://github.com/google/python-fire
[^4]: https://github.com/google/python-fire/blob/master/doc/guide.md
